<a href="https://colab.research.google.com/github/kowook137/Model_Discriminator_module/blob/main/Base_model_for_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#딥러닝 분류기 모델/ 딥러닝 검증 모듈 구현 코드
#전이학습을 위한 base 코드 작성

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [11]:
#데이터셋 로드
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR100(root='./data', train=False,download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

In [12]:
#Base model 정의(구조: Conv2D -> ReLU -> MaxPool -> FC -> Softmax)
class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 100)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

In [13]:
#학습 환경 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
base_model = BaseModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(base_model.parameters(), lr=0.001)

In [14]:
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = base_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f"[Epoch {epoch+1}] Loss: {running_loss:.3f}")

[Epoch 1] Loss: 2718.903
[Epoch 2] Loss: 2117.742
[Epoch 3] Loss: 1842.154
[Epoch 4] Loss: 1633.727
[Epoch 5] Loss: 1456.899
[Epoch 6] Loss: 1288.302
[Epoch 7] Loss: 1133.352
[Epoch 8] Loss: 987.134
[Epoch 9] Loss: 841.434
[Epoch 10] Loss: 719.086


In [15]:
torch.save(base_model.state_dict(), 'base_model.pth')
print("base_model 저장 완료: base_model.pth")

base_model 저장 완료: base_model.pth


In [16]:
#Base model이 train data 에 대해서 틀린 예측을 한 데이터 Dt_mis_inputs, Dt_mis_labels 를 추출하는 코드









In [17]:
#모델을 평가 모드로 설정함
base_model.eval()

BaseModel(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=4096, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=100, bias=True)
)

In [18]:
# 틀린 샘플 저장 리스트
misclassified_inputs = []
misclassified_labels = []

# 정답 클래스 수 계산용
total_samples = 0
incorrect_samples = 0

In [21]:
#base_model 이 틀린 예측을 한 데이터를 추출함.
with torch.no_grad():
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = base_model(inputs)
        _, predicted = torch.max(outputs, 1)

        incorrect_mask = predicted != labels
        if incorrect_mask.any():
            misclassified_inputs.append(inputs[incorrect_mask])
            misclassified_labels.append(labels[incorrect_mask])
            incorrect_samples += incorrect_mask.sum().item()
        total_samples += labels.size(0)


# 단일 tensor 로 병합함
Dt_mis_inputs = torch.cat(misclassified_inputs, dim=0)
Dt_mis_labels = torch.cat(misclassified_labels, dim=0)

#확인용 출력
print(f"총 훈련 샘플 수: {total_samples}")
print(f"틀린 예측 수: {incorrect_samples}")
print(f"Dt_mis shape: {Dt_mis_inputs.shape}, {Dt_mis_labels.shape}")

총 훈련 샘플 수: 50000
틀린 예측 수: 9184
Dt_mis shape: torch.Size([9184, 3, 32, 32]), torch.Size([9184])


In [22]:
# A 모델에서 틀린 샘플 저장함
torch.save(Dt_mis_inputs, 'Dt_mis_inputs.pt')
torch.save(Dt_mis_labels, 'Dt_mis_labels.pt')